# 0. Initialization

In [ ]:
parties_limit = 200
politicians_limit = 50

In [36]:
#Dependencies
#import numpy as np
import pandas as pd #data structures and conversion
import tweepy #access twitter API
import json
from pattern.text.nl import sentiment #sentiment analysis

In [37]:
#Tweepy
#twitter API tokens
token = "794396036-QbxWOP1J9x8ivv9R8n1oDduePHis5C9LiSUGRLkm"
token_secret = "VLE9H9ypvlL7O85Gynf3eWdczz7SW3Xgzy3Zq56ogXu0h"
key = "SB7lTkx5abtSzNui7eN6RpTLs"
secret = "mNkL5IsiVIQhCWF5qWqC8N8TJhyWPz1uAg2tfNcAamtYyPYJi3"

#Call Tweepy
auth = tweepy.OAuthHandler(key, secret)
auth.set_access_token(token, token_secret)
#api = tweepy.API(auth, parser=tweepy.parsers.JSONParser()) #bug
api = tweepy.API(auth)

# 1. Function defs

In [38]:
#Function extracts type of media from tweet, even if media is present in retweeted, quoted or replied tweet
#references can "go down" 2 tweets
def media(tweet, dict_): 
    #print("called")
    try:
    #extract media type from tweet if possible
        type = tweet.extended_entities['media'][0]['type']
        #print("media_checked")
    except:
    #failed, check if the tweet is a quote or reply, as these might contain media displayed to the user as well    
        if dict_["quoted_id"] or dict_["replied_id"]:
            if dict_["quoted_id"]:
                id = dict_["quoted_id"]
            elif dict_["replied_id"]:
                id = dict_["replied_id"]
            try:
                #get original tweet
                tweet2 = api.get_status(id, tweet_mode = 'extended')
            except:
                pass
            
            try:
            #extract media type from tweet if possible
                type = tweet2.extended_entities['media'][0]['type']
                #print("media_checked2")
            except:
            #failed, repeat process
                try:
                    id = tweet2.in_reply_to_status_id_str 
                except:
                    pass

                try:
                     id = tweet2.quoted_status_id_str
                except:
                    pass
                
                try:  
                    tweet3 = api.get_status(id, tweet_mode = 'extended')
                    type = tweet3.extended_entities['media'][0]['type']
                    #print("media_checked3")
                except:
                    type = 'nothing'
        else:
            type = None
    #print(type)
    return type

In [39]:
#Function iterates over all hashtags of a tweet and returns the number of trending hashtags that day
#Uses variable dictOfHashtags if possible, containing all the trending hashtags of previously searched days
#Fills dictOfHashtags with new date: trending hashtags pair if any
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import re
import time
try:
    data = json.load(open("hashtags.json", "r"))
    df = pd.DataFrame.from_dict(data, orient="index")
    dictOfHashtags = df.to_dict('dict')[0]
except:
    dictOfHashtags = {}
    
def hashtags(dict_, dictOfHashtags):
    
    dt = dict_['date']
    #print(dt)
    try:
        string = dictOfHashtags[dt]
    except:
        url = 'https://trendogate.com/placebydate/23424757/'+dt
        #print(url)
        response = requests.get(url)
        tx = BeautifulSoup(response.text, 'html.parser')
        string = ""
        #flag = true
        ind = tx.find("h2",text=" Trends in Belgium / Belgium ")
        while ind:
            ind = ind.find_next("li")
            try:
                string = string + ind.text
            except:
                #print("end of hashtags")
                pass
        string = re.sub('#', '', string)
        dictOfHashtags[dt]=string
        time.sleep(1)
    count = 0
    try:
        for el in dict_['hashtags']:
            #print(el['text'])
            if el['text'] in string: 
                count = count + 1
    except:
        pass
        #print("In function hashtags: exception for:"+dict_['id_str'])
    #print(string)
    #print (str(count)+'\n \n')
    return count, dictOfHashtags

In [40]:
#Extracts emojis from tweet
#Source: https://stackoverflow.com/questions/43146528/how-to-extract-all-the-emojis-from-text
import regex
import emoji
def emojis_str(text):

    emoji_list = []
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI for char in word):
            emoji_list.append(word)

    return ''.join(emoji_list)

# 2. Tweet downloader

In [41]:
#init
import datetime
from datetime import date
listOfTweets = [] #empty
parties = ["pvdabelgie", "groen", "sp_a", "cdenv", "openvld", "de_NVA","vlbelang"] #left to right
accounts = ["pvdabelgie", "peter_mertens", "RaoulHedebouw", "TomDMeester", "JosDHaese", "LiseVDC", "DeWitteKim", 
            "groen", "MeyremAlmaci","pdsutter","kristofcalvo", "WouterDeVriendt", "BjornRzoska", "TinneVdS",
            "sp_a", "johncrombez","kvanbrempt", "conner_rousseau","freyabos", "JorisVDBroucke","MohamedRidouani",
            "cdenv", "joachimcoens", "wbeke", "Koen_Geens1", "crevits", "dalle_benjamin", "ServaisV",
            "openvld", "RuttenGwendolyn","Maggie_DeBlock","alexanderdecroo","Barttommelein", "svengatz", "goedeleliekens",
            "de_NVA","Bart_DeWever", "Zu_Demir","FranckenTheo", "JanJambon", "LiesbethHomans","jvanovertveldt",
            "vlbelang", "FDW_VB","gannemans","DVanLangenhove", "SamvanRooy1","Barbara_Pas","chrisjanssensVB"] #left to right
#i = 0
#iterate over all parties
for acc in accounts:
    #amount of tweets to try to download per party
    if acc in parties:
        limit = parties_limit
    else:
        limit = politicians_limit
        
#iterate over #limit tweets
#MUST USE EXTENDED TWEET_MODE OR TEXT WILL BE TRUNCATED AND MEDIA NOT FOUND
    for tweet in tweepy.Cursor(api.user_timeline, screen_name=acc, tweet_mode="extended").items(limit):
        try:
            is_retweet = tweet.retweeted_status.id_str 
        except:
            is_retweet = ''
        if not is_retweet:
            dict_ = {"id_str": tweet.id_str,
                    "Screen Name": tweet.user.screen_name,
                     "User Name": tweet.user.name,
                     "Time": tweet.created_at,
                     "Followers": tweet.user.followers_count,
                     "num_retweets": tweet.retweet_count,
                     #"Retweeted": tweet.retweeted,
                     "num_likes": tweet.favorite_count,
                     #"Favorited": tweet.favorited,
                    }
            

            try: 
                #dict_["hashtags"] = tweet.entities.['hashtags']
                #print(tweet.entities['hashtags'])
                dict_['hashtags'] = tweet.entities['hashtags']
                dict_['num_hashtags'] = len(tweet.entities['hashtags'])
            except IndexError:
                dict_['hashtags'] = None
                dict_['num_hashtags'] = 0
            try: 
                dict_['mentions'] = tweet.entities['user_mentions']
                dict_['num_mentions'] = len(tweet.entities['user_mentions'])
                #dict_['mentions'] = tweet.entitiesuser_mentions
            except IndexError:
                dict_['mentions'] = None
                dict_['num_mentions'] = 0
            try:
                dict_['urls'] = tweet.extended_entities['urls']
                dict_['num_urls'] = len(tweet.entities['urls'])
            except:
                dict_['urls'] = None
                dict_['num_urls'] = 0
            #extract ID's of replied or quoted tweets
            #access to field different than next two!
            try:
                dict_['replied_id'] = tweet.in_reply_to_status_id_str
            except:
                dict_['replied_id'] = ''

            try:
                dict_['quoted_id'] = tweet.quoted_status_id_str
            except:
                dict_['quoted_id'] = ''

            #extract text
            #must be done this way to clearly extract emojis
            tweettemp = tweet._json 
            dict_['tweet_text'] = tweettemp['full_text']
            try:
                dict_['quote_text'] = tweettemp.quoted_status['full_text']
            except:
                dict_['quote_text'] = ''

                        
            #extracts media type via function call to media function
            #place here for easy access to id of quoted or replied tweet
            dict_['media_type'] = media(tweet, dict_)
                     
            time_tweet = tweet.created_at
            dict_['date'] = time_tweet.strftime("%Y-%m-%d")
            result = hashtags(dict_, dictOfHashtags)
            dict_['num_hashtags_trending'] = result[0]
            dictOfHashtags = result[1]
            
            result = sentiment(dict_['tweet_text'])
            dict_['polarity'] = result[0]
            dict_['subjectivity'] = result[1]
            print('replied:')
            print(tweet.in_reply_to_status_id_str)
            print('quoted:')
            print(dict_['quoted_id'])
            listOfTweets.append(dict_)

replied:
None
quoted:

replied:
None
quoted:
1210563130532802560
replied:
None
quoted:

replied:
None
quoted:

replied:
None
quoted:

replied:
None
quoted:

replied:
None
quoted:

replied:
1211932070039216128
quoted:

replied:
None
quoted:

replied:
None
quoted:

replied:
None
quoted:

replied:
None
quoted:

replied:
None
quoted:
1210023311154900992
replied:
None
quoted:

replied:
None
quoted:

replied:
None
quoted:

replied:
1206949540164751362
quoted:

replied:
None
quoted:

replied:
None
quoted:

replied:
None
quoted:
1207563850293399552
replied:
None
quoted:

replied:
None
quoted:

replied:
None
quoted:

replied:
None
quoted:

replied:
None
quoted:

replied:
1207636486830665728
quoted:

replied:
None
quoted:

replied:
None
quoted:

replied:
None
quoted:

replied:
None
quoted:

replied:
None
quoted:
1211980179574345728
replied:
None
quoted:
1211980179574345728
replied:
None
quoted:
1212466660138872832


In [42]:
print(listOfTweets)

[{'id_str': '1211222473477435393', 'Screen Name': 'peter_mertens', 'User Name': 'Peter Mertens', 'Time': datetime.datetime(2019, 12, 29, 9, 48, 35), 'Followers': 15579, 'num_retweets': 26, 'num_likes': 84, 'hashtags': [], 'num_hashtags': 0, 'mentions': [], 'num_mentions': 0, 'urls': None, 'num_urls': 0, 'replied_id': None, 'quoted_id': '', 'tweet_text': '“Het Belgische systeem is stervende. De status quo is niet meer mogelijk. Ofwel valt de boel uiteen. Dan gaat het land in 2024 naar de exit. Dat gaat massaal veel geld kosten. Ofwel moet België versterkt worden. Maar dan moeten we dit land hertimmeren.”\nhttps://t.co/xPQompzVLg', 'quote_text': '', 'media_type': 'nothing', 'date': '2019-12-29', 'num_hashtags_trending': 0, 'polarity': -0.11666666666666664, 'subjectivity': 0.6833333333333332}, {'id_str': '1210564131083960323', 'Screen Name': 'JosDHaese', 'User Name': "Jos D'Haese", 'Time': datetime.datetime(2019, 12, 27, 14, 12, 34), 'Followers': 3408, 'num_retweets': 20, 'num_likes': 78,

In [55]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

data = pd.DataFrame(listOfTweets)

data['reply'] = pd.notnull(data['replied_id'])
data['quote'] = data['quoted_id'].ne('')
data['tweet_type'] = 'normal'
data.loc[data['reply'].eq(True), 'tweet_type'] = 'reply'
data.loc[data['quote'].eq(True), 'tweet_type'] = 'quote'

#numerical
emojis = data['tweet_text'].apply(emojis_str)
data['num_emojis'] = emojis.apply(len)
scores = emojis.apply(analyser.polarity_scores)
data['emoji_polarity'] = scores.apply(pd.Series)['compound']

data['num_urls'] = data['tweet_text'].str.count("http") #quote counts for 1 link

data['length'] =data['tweet_text'].apply(len)

#place people in correct party
pvdabelgie = data['Screen Name'].eq("pvdabelgie")  | data['Screen Name'].eq("peter_mertens") | data['Screen Name'].eq("RaoulHedebouw") | data['Screen Name'].eq("TomDMeester") | data['Screen Name'].eq("JosDHaese") | data['Screen Name'].eq("LiseVDC") | data['Screen Name'].eq("DeWitteKim")
groen = data['Screen Name'].eq("groen") | data['Screen Name'].eq("MeyremAlmaci") | data['Screen Name'].eq("pdsutter") | data['Screen Name'].eq("kristofcalvo") | data['Screen Name'].eq("WouterDeVriendt") | data['Screen Name'].eq("BjornRzoska") | data['Screen Name'].eq("TinneVdS")
sp_a = data['Screen Name'].eq("sp_a") | data['Screen Name'].eq("johncrombez") | data['Screen Name'].eq("kvanbrempt") | data['Screen Name'].eq("conner_rousseau") | data['Screen Name'].eq("freyabos") | data['Screen Name'].eq("JorisVDBroucke") | data['Screen Name'].eq("MohamedRidouani")
cdenv = data['Screen Name'].eq("cdenv") |  data['Screen Name'].eq("joachimcoens") | data['Screen Name'].eq("wbeke") | data['Screen Name'].eq("Koen_Geens1") | data['Screen Name'].eq("crevits") | data['Screen Name'].eq("dalle_benjamin") | data['Screen Name'].eq("ServaisV")
openvld = data['Screen Name'].eq("openvld") | data['Screen Name'].eq("RuttenGwendolyn") | data['Screen Name'].eq("Maggie_DeBlock") | data['Screen Name'].eq("alexanderdecroo") | data['Screen Name'].eq("Barttommelein") | data['Screen Name'].eq("svengatz") | data['Screen Name'].eq("goedeleliekens")
de_NVA = data['Screen Name'].eq("de_NVA") | data['Screen Name'].eq("Bart_DeWever") | data['Screen Name'].eq("Zu_Demir") | data['Screen Name'].eq("FranckenTheo") | data['Screen Name'].eq("JanJambon") | data['Screen Name'].eq("LiesbethHomans") | data['Screen Name'].eq("jvanovertveldt")
vlbelang = data['Screen Name'].eq("vlbelang") | data['Screen Name'].eq("FDW_VB") | data['Screen Name'].eq("gannemans") | data['Screen Name'].eq("DVanLangenhove") | data['Screen Name'].eq("SamvanRooy1") | data['Screen Name'].eq("Barbara_Pas") | data['Screen Name'].eq("chrisjanssensVB") | data['Screen Name'].eq("FDW_VB, ")
data.loc[pvdabelgie == True, 'screen_name'] = 'pvdabelgie'
data.loc[groen == True, 'party_affiliation'] = 'groen'
data.loc[sp_a == True, 'party_affiliation'] = 'sp_a'
data.loc[cdenv == True, 'party_affiliation'] = 'cdenv'
data.loc[openvld == True, 'party_affiliation'] = 'openvld'
data.loc[de_NVA == True, 'party_affiliation'] = 'de_NVA'
data.loc[vlbelang == True, 'party_affiliation'] = 'vlbelang'

data.loc[data['media_type'].isnull(), 'media_type'] = 'nothing'
data.loc[data['media_type'].eq('photo') | data['media_type'].eq('animated gif'), 'media_type'] = 'photo'

time_tweet = pd.to_datetime(data['Time'], format='%Y-%m-%d %H:%M:%S')
delta = time_tweet.dt.date-date.today() 
data['days_since_post'] = -delta.dt.days
data['time_day'] = time_tweet.dt.hour
data['day_week'] = time_tweet.dt.dayofweek

data['time'] = 'night'
data.loc[data['time_day'].ge(6) | data['time_day'].le(12), 'time'] = 'morning'
data.loc[data['time_day'].ge(12) | data['time_day'].le(18), 'time'] = 'day'
data.loc[data['time_day'].ge(18) | data['time_day'].le(14), 'time'] = 'evening'

data['weekend'] = False
data.loc[data['day_week'].eq(5) | data['day_week'].eq(6), 'weekend'] = True

In [56]:
print(data['replied_id'] + str(data['reply']))

0                                                   NaN
1                                                   NaN
2                                                   NaN
3                                                   NaN
4                                                   NaN
5                                                   NaN
6                                                   NaN
7     12119320700392161280     False\n1     False\n2...
8                                                   NaN
9                                                   NaN
10                                                  NaN
11                                                  NaN
12                                                  NaN
13                                                  NaN
14                                                  NaN
15                                                  NaN
16    12069495401647513620     False\n1     False\n2...
17                                              

# 3. Save data
In JSON and csv

In [57]:
#save in JSON
#TempJSON = json.dumps(listOfTweets, indent=4, sort_keys=True, default=str)
#f = open("tweets.json", "w")
data.to_json('tweets_scrapetest.json')
print("json saved")

#save in standard csv
#TempDF = pd.DataFrame.from_dict(listOfTweets)
data.to_csv('tweets_scrapetest.csv')
print("csv saved")

json saved
csv saved


In [46]:
#T = pd.DataFrame.from_dict(dictOfHashtags)
#T.to_csv('dictOfHashtags.csv')
#print("hashtags saved")

In [47]:
print(dictOfHashtags)

{'2019-04-25': ' health bigdata Security E411 Liège bond25 FutureofEurope liveonlinedialogue deafspraak buitenspeeldag antsta terzaketv nuclear BBMAsTopSocial TPMP Migration statbel MUNMCI WorldMalariaDay SETT VIBSeminar19 deochtend flechewallonne Endgame connect2change European Parliament degroof petercam kamer Emmanuel Macron KV Mechelen Corneel flandre Dick Rivers Vlaams Belang De Wever Parlement italy Palestijnen marvel Broechem Joe Biden Jean-Pierre Marielle Pogba ThisTimeImVoting neufchâteau VaccinesWork EUelections2019 AvengersEndgame Uber conferencedepresse', '2019-04-26': ' begov Japan sustainability nuclear Broechem health startup brf2019 COYM justitie Migration NieuweStrijd climat lgbtqastro afschakelplan Daniel AzerbaijanGP DataViz Endgame lagranderassrah4 brussels cluand E411 thistimeimvoting DaysGone rawpolitics dewegvooruit SETT EUelections2019 VaccinesWork Neufchâteau Président Bart De Wever Vlaamse Vlaams Belang Days Gone Open VLD Maggie De Block Daniël Avengers Joe Bi

In [48]:
TempJSON = json.dumps(dictOfHashtags, indent=4, sort_keys=True, default=str)
f = open("hashtags.json", "w")
f.write(TempJSON)
f.close
print("hashtags saved")

hashtags saved


In [49]:
data = json.load(open("hashtags.json", "r"))
df = pd.DataFrame.from_dict(data, orient="index")
dict_hash = df.to_dict('dict')[0]
for el in dict_hash:
    print(el)

2019-04-25
2019-04-26
2019-05-01
2019-05-03
2019-05-04
2019-05-05
2019-05-06
2019-05-07
2019-05-09
2019-05-13
2019-05-15
2019-05-16
2019-05-17
2019-05-21
2019-05-22
2019-05-23
2019-05-25
2019-05-26
2019-06-06
2019-06-28
2019-07-01
2019-07-02
2019-07-10
2019-07-11
2019-07-19
2019-07-26
2019-07-30
2019-08-09
2019-08-10
2019-08-13
2019-08-20
2019-08-27
2019-08-31
2019-09-03
2019-09-04
2019-09-12
2019-09-13
2019-09-21
2019-09-23
2019-09-25
2019-09-28
2019-09-30
2019-10-01
2019-10-02
2019-10-04
2019-10-07
2019-10-08
2019-10-09
2019-10-10
2019-10-14
2019-10-16
2019-10-17
2019-10-18
2019-10-19
2019-10-21
2019-10-22
2019-10-23
2019-10-24
2019-10-25
2019-10-27
2019-10-29
2019-10-31
2019-11-01
2019-11-02
2019-11-03
2019-11-04
2019-11-05
2019-11-06
2019-11-07
2019-11-08
2019-11-09
2019-11-10
2019-11-11
2019-11-12
2019-11-13
2019-11-14
2019-11-15
2019-11-16
2019-11-17
2019-11-18
2019-11-19
2019-11-20
2019-11-21
2019-11-22
2019-11-23
2019-11-24
2019-11-25
2019-11-26
2019-11-27
2019-11-28
2019-11-29

In [50]:
print(dict_hash['2019-04-25'])

 health bigdata Security E411 Liège bond25 FutureofEurope liveonlinedialogue deafspraak buitenspeeldag antsta terzaketv nuclear BBMAsTopSocial TPMP Migration statbel MUNMCI WorldMalariaDay SETT VIBSeminar19 deochtend flechewallonne Endgame connect2change European Parliament degroof petercam kamer Emmanuel Macron KV Mechelen Corneel flandre Dick Rivers Vlaams Belang De Wever Parlement italy Palestijnen marvel Broechem Joe Biden Jean-Pierre Marielle Pogba ThisTimeImVoting neufchâteau VaccinesWork EUelections2019 AvengersEndgame Uber conferencedepresse
